In [27]:
import numpy as np
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import string

from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [28]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [37]:
#Loading the file location
path = 'Gullivers_Travel'
text = open(path, encoding ='utf').read().lower()
text= text.translate(str.maketrans('', '', string.punctuation))

In [38]:
# # #Removing undesirable words and characters
# # print ('Initial---------')
# # print('Corpus Length', len(text))
# # chars = sorted(list(set(text)))
# # print('Total character types:', len(chars))
# # bad_words = ['www.xtratutors.com']
# # with open('pg14916.txt') as oldfile, open('pg14916_new.txt', 'w') as newfile:
# #     for line in oldfile:
# #         if not any(bad_word in line for bad_word in bad_words):
# #             line= line.translate(str.maketrans('','',"\n"string.punctuation))
# #             newfile.write(line)
            
# # path = 'pg14916_new.txt'
# # text = open(path).read().lower()
# print ('Final---------')
# print('Corpus Length', len(text))
# chars = sorted(list(set(text)))
# print('Total character types:', len(chars))

In [40]:
chars = sorted(list(set(text)))
#Define Dictionary For Converting to One Hot.
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [41]:
#Converting to the trainble matrix size
maxlen = 100
step = 1
lettergroup = []
nextletter = []
for i in range(0, len(text) - maxlen, step):
    lettergroup.append(text[i: i + maxlen])
    nextletter.append(text[i + maxlen])
print('nb sequences:', len(lettergroup))

nb sequences: 552095


In [42]:
#Convert to One Hot
X = np.zeros((len(lettergroup), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(lettergroup), len(chars)), dtype=np.bool)
for i, group in enumerate(lettergroup):
    for t, char in enumerate(group):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[nextletter[i]]] = 1

In [43]:
# #Building The LSTM Model
# #model1
# model = Sequential()
# model.add(LSTM(512, input_shape=(maxlen, len(chars)),return_sequences=True))
# model.add(Dropout(0.5))
# model.add(LSTM(512))
# model.add(Dropout(0.5))
# model.add((Dense(len(chars), activation='softmax')))
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [44]:
#Building The LSTM Model
#model2
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add((Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [45]:
filepath="weights-improvement-{epoch:02d}-{loss:4f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='loss',verbose=1,save_best_only=True, mode='min')
callbacks_list=[checkpoint]

In [46]:
#Temperature...Helper function to sample an index from a probability array...
def sample(preds, temperature=1):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
#Model Training And Output Generation After Each Step
epochstot=20
for iteration in range(1, epochstot):
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128, epochs=1, callbacks=callbacks_list)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
552095/552095 [==============================] - 2002s - loss: 1.8798  

----- diversity: 0.2
----- Generating with seed: " just daylight  i attempted to rise but was not able to stir for as i happened to lie on my back i f"
 just daylight  i attempted to rise but was not able to stir for as i happened to lie on my back i for the strong and the strong the strong and the strong and the strong and the strong and the strong the strong of the strong my hand of the strong of the strenges of the strect of the strong and those which we had a served the strong the strong the strong of the strong of the streng of the strong that i had not be a should be signs of the sea of the strong of the strong of the streng and the stron

----- diversity: 0.5
----- Generating with seed: " just daylight  i attempted to rise but was not able to stir for as i happened to lie on my back i f"
 just daylight  i attempted to rise but was not 

In [ ]:
# #Call a trained model
# filename = "/home/nick/Desktop/My Space/Wonderland/weights-improvement-00-3.009437.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer=optimizer)